In [5]:
import os
from openai import OpenAI
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

if not os.getenv("OPENAI_API_KEY"):
    print("FATAL ERROR: OPENAI_API_KEY environment variable is not set.")
else:
    print("OpenAI API key found.")
    client = OpenAI()

OpenAI API key found.


In [6]:
system_prompt = """You are a snarky assistant that analyzes the contents of a website,
and provides a short, snarky, humorous summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.

WEBSITE CONTENT:
"""

In [9]:
def fetch_website_contents(url):
    print(f"Fetching content from {url} using Selenium...")
    driver = None
    
    try:
        print("1. Setting up Chrome options...")
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("window-size=1200x600")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")

        print("2. Installing/finding WebDriver...")
        driver_path = ChromeDriverManager().install()
        print(f"WebDriver is at: {driver_path}")
        service = Service(driver_path)
        
        print("3. Initializing WebDriver...")
        driver = webdriver.Chrome(service=service, options=chrome_options)
        
        print("4. Setting page load timeout...")
        driver.set_page_load_timeout(10)

        print(f"5. Getting URL: {url}...")
        driver.get(url)
        
        print("6. Setting implicit wait...")
        driver.implicitly_wait(5)
        
        print("7. Finding body content...")
        body_content = driver.find_element(By.TAG_NAME, "body").text
        
        print("Content fetched successfully.")
        return body_content
    
    except Exception as e:
        print(f"Error during Selenium operation: {e}")
        if driver:
            body_content = driver.find_element(By.TAG_NAME, "body").text
            if body_content:
                print("Page timed out, but returning partial content.")
                return body_content
        return None
    finally:
        if driver:
            print("8. Shutting down WebDriver...")
            driver.close()
            driver.quit()
            print("WebDriver shut down.")

In [10]:
def summarize(url):
    website_content = fetch_website_contents(url)
    
    if not website_content:
        return "Sorry, I couldn't fetch the website. It's probably broken or my robot eyes can't see it."

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website_content}
    ]

    try:
        print("9. Calling OpenAI API...")
        response = client.chat.completions.create(
            model="gpt-4-turbo",
            messages=messages
        )
        print("OpenAI call successful.")
        return response.choices[0].message.content
    
    except Exception as e:
        print(f"Error calling OpenAI: {e}")
        return "Error from AI: It probably got bored and left."

In [11]:
target_url = "https://example.com"
snarky_summary = summarize(target_url)

print("\n--- SNARKY SUMMARY ---")
print(snarky_summary)

Fetching content from https://example.com using Selenium...
1. Setting up Chrome options...
2. Installing/finding WebDriver...
WebDriver is at: /Users/jim/.wdm/drivers/chromedriver/mac64/143.0.7499.169/chromedriver-mac-arm64/chromedriver
3. Initializing WebDriver...
4. Setting page load timeout...
5. Getting URL: https://example.com...
6. Setting implicit wait...
7. Finding body content...
Content fetched successfully.
8. Shutting down WebDriver...
WebDriver shut down.
9. Calling OpenAI API...
OpenAI call successful.

--- SNARKY SUMMARY ---
Ah, Example Domain, the digital equivalent of a showroom mannequin. It's just there to look pretty in tutorials and docs, making sure you know not to actually use it for real-world chaos. No real news here, unless "don't use this in production" is a revelation!


In [12]:
system_prompt = """You are a  assistant that analyzes the contents of a website,
and provides a short and concise summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.

WEBSITE CONTENT:
"""

In [14]:
from pydantic import networks


target_url = "https://www.offthepress.com/"
news_summary = summarize(target_url)

print("\n--- NEWS SUMMARY ---")
print(news_summary)

Fetching content from https://www.offthepress.com/ using Selenium...
1. Setting up Chrome options...
2. Installing/finding WebDriver...
WebDriver is at: /Users/jim/.wdm/drivers/chromedriver/mac64/143.0.7499.169/chromedriver-mac-arm64/chromedriver
3. Initializing WebDriver...
4. Setting page load timeout...
5. Getting URL: https://www.offthepress.com/...
6. Setting implicit wait...
7. Finding body content...
Content fetched successfully.
8. Shutting down WebDriver...
WebDriver shut down.
9. Calling OpenAI API...
OpenAI call successful.

--- NEWS SUMMARY ---
### Summary of Latest News and Developments

**Key Announcements:**
- **Federal Holidays:** Trump signs an executive order designating Christmas Eve and the day after Christmas as federal holidays.
- **Medicare Changes:** Starting next year, enrollees will pay 50% less for some drugs, according to a recent study.
- **Drug Policy Reform:** The Trump administration reclassifies cannabis through an executive order.
- **Innovation and Ce